## Overview

This script reproduces the main analyses and figures in the paper "MEGaNorm: Normative Modeling of MEG Brain Oscillations Across the Human Lifespan". We demonstrates how to derive **normative models** of Magnetoencephalography (MEG) recordings and show their application. To do so, we use ([MEGaNorm package](https://pypi.org/project/meganorm/)).

The workflow starts with reading MEG data, preprocessing, and extracting relevant features. We then compare two **Hierarchical Bayesian Regression (HBR)** models: **non-linear**, **heteroscedastic**, and **non-Gaussian** model  versus **linear**, **homoscedastic**, and **Gaussian** model. To evaluate these models, we run the normative modeling process **10 times** using different train-test splits (generated with different random seeds), and compute a range of evaluation metrics.

After selecting the best-performing model, we demonstrate its predictive power by using it to classify Parkinson's disease. We also explore Parkinson’s disease as a spectrum of abnormalities by analyzing **deviation scores** to decode heterogeneity of this disease. Finally, we estimate normative models on the full dataset and generate **growth charts**, which are later used to create: **Population-based Neuro-Oscillatory Charts (P-NOCs)** and **Individual-based Neuro-Oscillatory Charts (I-NOCs)**.

> **Note:** This notebook runs some steps in **parallel**.  
> For a sequential version, see `MEG_Norm_sequential.ipynb`.

---

## Notebook Workflow

1. **Feature extraction.**

2. **Comparison of non-linear, heteroscedastic, and non-Gaussian HBR model with linear, homoscedastic, and Gaussian HBR model.**

3. **Using deviation scores to detect Parkinson's disease from healthy participants.**

4. **Decoding heterogeneity in Parkinson's disease via theta-beta deviation scores.**

5. **From growth charts to P-NOCs and I-NOCs.**


Each step is explained in detail throughout the notebook.

---

## Parameters to Specify

- **`datasets`**  
  A nested Python dictionary where each key corresponds to a dataset name (e.g., `'BTNRH'`, `'CAMCAN'`, etc.). Each value is a dictionary with the following fields:

  - **`base_dir`**:  
    The base directory path for the dataset, typically the root folder containing the subject data.

  - **`task`**:  
    The task label used in the filenames (e.g., `'task-rest'` in `sub-xx_ses-02_task-rest_run-02_meg.ds`). Note that files must be named according to the BIDS format.

  - **`ending`**:  
    The filename suffix, including its extension (e.g., `'meg.fif'`, `'meg.ds'`, or `'4-Restin/4D'`). Note that files must be named according to the BIDS format.

  This structure ensures compatibility across various MEG hardware systems.

- **`project_dir`**  
  The directory where all output files will be saved.  
  **Note:** Use a consistent path across all notebooks to avoid confusion or file overwriting.

- **`conda_environment_name`**  
  The name of the Python environment used to run the jobs.  
  This environment must have the **MEGaNorm** package installed.

- **`slurm_username`**  
  If using SLURM for parallel computation, provide your SLURM `username`.  
  This enables job monitoring and automatic resubmission of incomplete tasks.


In [1]:
datasets = {
    'BTNRH': {
        'base_dir': ... ,
        'task': "task-rest",
        "ending" : "meg.fif"
        },
    'CAMCAN': {
        'base_dir': ... ,
        'task': "task-rest",
        "ending" : "meg.fif"
        },
    'NIMH': {
        'base_dir': ... ,
        'task': "task-rest",
        "ending" : "meg.ds"
        },
    'OMEGA': {
        'base_dir': ... ,
        'task': "task-rest",
        "ending" : "meg.ds"
        },
    'HCP': {
        'base_dir': ... ,
        'task': "task-rest",
        "ending" : "4-Restin/4D"
        },
    'MOUS': {
        'base_dir': ... ,
        'task': "task-rest",
        "ending" : "meg.ds"
    }
    }

project_dir = ...
conda_environment_name = ...
slurm_username = ...

# import packages

In [ ]:
import meganorm

In [3]:
# === Standard library ===
import os
import sys
import json
import pickle
import warnings
import tempfile
import multiprocessing

# === Third-party libraries ===
import numpy as np
import pandas as pd

# === External tools ===
from pcntoolkit.normative_parallel import execute_nm, collect_nm

# === meganorm: I/O utilities ===
from meganorm.utils.IO import (
    merge_fidp_demo,
    merge_datasets_with_glob,
    factorize_columns,
    normalize_column,
    separate_patient_data,
    make_config
)

# === meganorm: parallel tools ===
from meganorm.utils.parallel import (
    submit_jobs,
    check_jobs_status,
    collect_results,
    auto_parallel_feature_extraction
)

# === meganorm: normative modeling utilities ===
from meganorm.utils.nm import (
    hbr_data_split,
    estimate_centiles,
    abnormal_probability,
    calculate_PNOCs,
    prepare_prediction_data,
    cal_stats_for_INOCs,
    aggregate_metrics_across_runs,
)

# === Optional or legacy plotting tools ===
from meganorm.plots.plots import (
    z_scores_scatter_plot,
    plot_INOCs,
    plot_metrics,
    plot_PNOCs,
    qq_plot,
    box_plot_auc,
    plot_extreme_deviation,
    plot_growthcharts
)

# === Ignore warnings ===
warnings.filterwarnings("ignore")


**Making the necessary folders and directories to save the features, models and pictures**

In [4]:
features_dir = os.path.join(project_dir, 'Features')
features_log_path = os.path.join(features_dir, 'log')
features_temp_path = os.path.join(features_dir,'temp')
figures_dir = os.path.join(project_dir, "figures")

# configs for parallel feature extraction
job_configs = {'log_path':features_log_path, 'module':conda_environment_name, 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

# a log file to save logs of feature extraction
if not os.path.isdir(features_log_path):
    os.makedirs(features_log_path)

# a directory to save extracted features temporarily
if not os.path.isdir(features_temp_path):
    os.makedirs(features_temp_path)

# To save figures
if not os.path.isdir(figures_dir):
    os.makedirs(figures_dir)

# Create a config file that contains all parameters for feature extraction,  
# including filtering settings, which features to extract, and other options.
configs = make_config(project_dir)

subjects = merge_datasets_with_glob(datasets)

# 1. **Feature extraction**
The feature extraction process includes reading the data, preprocessing, periodic isolation, feature extraction, and summarization across the brain. This is handled by the `mainParallel` module from the **MEGaNorm** package, which supports both parallel and sequential execution. In this notebook, we use the parallel mode.

The features include:
- Relative power of theta
- Relative power of alpha
- Relative power of beta
- Relative power of gamma

In [ ]:
auto_parallel_feature_extraction(
    mainParallel_path=os.path.abspath(meganorm.src.mainParallel.__file__),
    features_dir=features_dir,
    subjects=subjects,
    job_configs=job_configs,
    config_file=os.path.join(project_dir, 'configuration.json'),
    username=slurm_username,
    auto_rerun=True,
    auto_collect=True,
    max_try=3)

# 2. **Comparison of non-linear, heteroscedastic, and non-Gaussian HBR model with linear, homoscedastic, and Gaussian HBR model.**

Here, we compare two HBR models to select the best one for further analysis. Models are estimated on the training data and evaluated on hold-out test data using a range of performance metrics.

To better assess the **generalization error** and **robustness** of the models, this process is repeated **10 times**, each using different train/test splits generated by distinct random seeds. For each split, the feature matrix `X`, target variable `Y`, and batch effect values are saved in a format compatible with **PCNToolkit**. The HBR models are then trained and tested across these runs, and the resulting metrics are stored for visualization.

The evaluation metrics include:

- **Standardized Mean Squared Error (SMSE)**
- **Skewness**
- **Excess Kurtosis**
- **Shapiro–Wilk test statistic**
- **Mean Absolute Centile Error (MACE)**

These metrics are visualized to summarize and compare model performance.


### Defining Random Seeds

To ensure model generalizability, we generate 10 distinct train–test splits by using different random seeds. Below, we define a list of `random_seeds` to create these splits.


In [5]:
random_seeds = [42, 100, 0, 10, 12, 73, 50, 9, 30, 51]

**Data preperation**

In [6]:
features_dir = os.path.join(project_dir, 'Features')
figures_dir = os.path.join(project_dir, "figures")
nm_processing_dir = os.path.join(project_dir, 'NM', 'Run_' + str(0))

### Data preparation for Normative Modeling
data_base_dirs = [values["base_dir"] for values in datasets.values()]
dataset_names = list(datasets.keys())

# Merge demographic data and extracted f-IDPS
df = merge_fidp_demo(datasets_paths=data_base_dirs,
        features_dir=features_dir,
        dataset_names=dataset_names,
        drop_columns = ["eyes"])

# Factorize categorical variables
df = factorize_columns(df, 
        columns=["sex", "site"])

# Scale age column by dividing by 100
df = normalize_column(df,
        column="age",
        normalizer=100)

# Separate patients and healthy participants
df_healthy, df_patients = separate_patient_data(df, 
        diagnosis=["parkinson"])

# # Train-Test split for the number of runs
for run_counter, run_id in enumerate(random_seeds):
        nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{run_counter}") 
                
        biomarker_names = hbr_data_split(df_healthy, 
                nm_processing_dir_temp, 
                drop_nans=True, 
                batch_effects=['sex', 'site'], 
                random_seed=run_id, 
                train_split=0.5, 
                stratification_columns=["site"])

**HBR Configs**

In [7]:
python_path = os.path.realpath(sys.executable)

hbr_configs = {
                'homo_Gaussian_linear':{'model_type':'linear', 
                                        'likelihood':'Normal', 
                                        'linear_sigma':'False',
                                        'random_slope_mu':'False', 
                                        'linear_epsilon':'False', 
                                        'linear_delta':'False'}, 

                'hetero_SHASH_bspline':{'model_type':'bspline', 
                                        'likelihood':'SHASHb', 
                                        'linear_sigma':'True',
                                        'random_slope_mu':'False', 
                                        'linear_epsilon':'True', 
                                        'linear_delta':'True'},
            }

inscaler='None' 
outscaler='None' 
batch_size = 1
outputsuffix = '_estimate'

memory = '2gb'
duration = '5:00:00'
cluster_spec = 'slurm'

**Run models n (= lenght of random seeds) times**

In [ ]:
for method in hbr_configs.keys():

    os.environ["PYTENSOR_FLAGS"] = f"compiledir={tempfile.mkdtemp(prefix='pytensor_compiledir_')}"
    processes = []

    for run_counter, run_id in enumerate(random_seeds):

        nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{run_counter}")

        respfile = os.path.join(nm_processing_dir_temp, 'y_train.pkl')
        covfile = os.path.join(nm_processing_dir_temp, 'x_train.pkl')
        trbefile = os.path.join(nm_processing_dir_temp, 'b_train.pkl')

        testrespfile_path = os.path.join(nm_processing_dir_temp, 'y_test.pkl')
        testcovfile_path = os.path.join(nm_processing_dir_temp, 'x_test.pkl')
        tsbefile = os.path.join(nm_processing_dir_temp, 'b_test.pkl')

        processing_dir = os.path.join(nm_processing_dir_temp, method) + '/'
        nm_log_path = os.path.join(processing_dir, 'log') + '/'

        if not os.path.isdir(processing_dir):
            os.makedirs(processing_dir)
        if not os.path.isdir(nm_log_path):
            os.makedirs(nm_log_path)

        p = multiprocessing.Process(target=execute_nm, 
            args = (processing_dir, 
                    python_path, 
                    'NM', 
                    covfile, 
                    respfile, 
                    batch_size, 
                    memory, 
                    duration),
            kwargs={
                'alg': 'hbr',
                'log_path': nm_log_path,
                'binary': True,
                'testcovfile_path': testcovfile_path,
                'testrespfile_path': testrespfile_path,
                'trbefile': trbefile,
                'tsbefile': tsbefile,
                'model_type': hbr_configs[method]['model_type'],
                'likelihood': hbr_configs[method]['likelihood'],
                'linear_sigma': hbr_configs[method]['linear_sigma'],
                'random_slope_mu': hbr_configs[method]['random_slope_mu'],
                'linear_epsilon': hbr_configs[method]['linear_epsilon'],
                'linear_delta': hbr_configs[method]['linear_delta'],
                'savemodel': 'True',
                'inscaler': inscaler,
                'outscaler': outscaler,
                'outputsuffix': outputsuffix,
                'interactive': 'auto',
                'cluster_spec': cluster_spec}
            )

        p.start()
        processes.append(p)

    for p in processes:
        p.join()    


**Aggregate all the models across runs**

In [ ]:
for method in hbr_configs.keys():
    for run_counter, run_id in enumerate(random_seeds):

        nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{run_counter}")
        processing_dir = os.path.join(nm_processing_dir_temp, method) + '/'

        collect_nm(processing_dir, "NM", collect=True, binary=True, batch_size=1)

**Calulating metrics across runs and saving them in a dictionary**

In [ ]:
testrespfile_path = os.path.join(nm_processing_dir, 'y_test.pkl')
testcovfile_path = os.path.join(nm_processing_dir, 'x_test.pkl')
tsbefile = os.path.join(nm_processing_dir, 'b_test.pkl')

which_metrics = ['SMSE', 'skewness', 'kurtosis', "W", "MACE"]

for method in hbr_configs.keys():
    
    metrics_values = aggregate_metrics_across_runs(
        path=nm_processing_dir,
        method_name=method,
        biomarker_names=biomarker_names,
        valcovfile_path=testcovfile_path,
        valrespfile_path=testrespfile_path,  
        valbefile=tsbefile,
        metrics = which_metrics,
        num_runs = 10,
        quantiles = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99],
        outputsuffix = "estimate",
        zscore_clipping_value = 8.0,
        )

    metrics_summary_path = os.path.join(os.path.join(project_dir, 'NM', f"metrics_summary_{method}.pkl"))
    with open(metrics_summary_path, "wb") as file:
        pickle.dump(metrics_values, file)

**Plot metrics across models**

In [ ]:
metrics_summary_path = [os.path.join(project_dir, 'NM', f"metrics_summary_{method}.pkl") for method in hbr_configs.keys()]

save_metrics_plots = os.path.join(figures_dir, "model_metrics")
if not os.path.isdir(save_metrics_plots):
    os.mkdir(save_metrics_plots)

plot_metrics(
    metrics_path=metrics_summary_path,
    which_biomarkers=biomarker_names,
    biomarkers_new_name=["Theta", "Alpha", "Beta", "Gamma"],
    colors=["teal", "orange"],
    save_path=save_metrics_plots,
            )

**Q-Q plot on preficted z-scores**

In [ ]:
processing_dir_temp = [os.path.join(nm_processing_dir, method) for method in hbr_configs.keys()]

save_metrics_plots = os.path.join(figures_dir, "model_metrics")
if not os.path.isdir(save_metrics_plots):
    os.mkdir(save_metrics_plots)

label_dict = {"Theta": 0, "Alpha": 1, "Beta": 2, "Gamma": 3}
qq_plot(processing_dir=processing_dir_temp,
        save_fig=save_metrics_plots,
        label_dict=label_dict,
        colors=["orange", "teal", "olive", "tomato"])


### Choosing the best model 

Based on the evaluation plots, the **non-linear**, **heteroscedastic**, and **non-Gaussian** HBR model consistently outperforms the baseline (**linear**, **homoscedastic**, and **Gaussian**) model.

Therefore, we select the non-linear HBR model as our final model and proceed with it in the subsequent analyses.


In [24]:
method = 'hetero_SHASH_bspline'

# 3. **Using deviation scores to detect Parkinson's disease from healthy participants.**

After selecting the best model, we use it to detect deviation scores for Parkinson's disease (PD) patients. Specifically, we apply the estimated normative models across runs to compute **deviation scores** for PD participants from the OMEGA dataset, and compare them to healthy participants in the test set of OMEGA.

Deviation scores for the healthy participants were already computed in **Step 2**, so we reuse those and only estimate deviation scores for the PD patients.

Once all scores are obtained, we compute **AUC (Area Under the Curve)** scores and assess their statistical significance using **permutation testing**, with correction for the **multiple comparisons problem**. Then, AUC values are plotted across **canonical frequency bands** to visualize classification performance. Finally, for further exploration of results, we apply **extreme deviation statistics**.

**Test on clinical data**

In [ ]:
# random_seeds = [0]
for run_counter, run_id in enumerate(random_seeds):

    nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{run_counter}")
    processing_dir_temp = os.path.join(nm_processing_dir_temp, method) + '/'
    nm_log_path = os.path.join(processing_dir_temp, 'log') + '/'

    prefix = "clinicalpredict_"
    prepare_prediction_data(df_patients.drop('diagnosis', axis=1),
                                nm_processing_dir_temp, 
                                drop_nans=True, 
                                batch_effects=['sex', 'site'], 
                                prefix=prefix)

    testrespfile_path = os.path.join(nm_processing_dir_temp, prefix + 'y_test.pkl')
    testcovfile_path = os.path.join(nm_processing_dir_temp, prefix + 'x_test.pkl')
    tsbefile = os.path.join(nm_processing_dir_temp, prefix + 'b_test.pkl')

    execute_nm(processing_dir_temp, 
    python_path,
            'NM', 
            testcovfile_path, 
            testrespfile_path, 
            batch_size, 
            memory, 
            duration, 
            alg='hbr', 
            log_path=nm_log_path, 
            binary=True, 
            tsbefile=tsbefile, 
            func="predict", 
            model_type=hbr_configs[method]['model_type'], 
            likelihood=hbr_configs[method]['likelihood'], 
            linear_sigma=hbr_configs[method]['linear_sigma'], 
            random_slope_mu=hbr_configs[method]['random_slope_mu'],
            linear_epsilon=hbr_configs[method]['linear_epsilon'], 
            linear_delta=hbr_configs[method]['linear_delta'], 
            savemodel='True', 
            inscaler=inscaler, 
            outscaler=outscaler, 
            outputsuffix="clinicalpredict", 
            inputsuffix=outputsuffix,
            interactive='auto', 
            cluster_spec=cluster_spec, 
            nuts_sampler="nutpie", 
            n_cores_per_batch="2")

**Calculating AUC scores and their significance**

In [ ]:
omega_site_id = np.where(np.asarray(dataset_names) == "OMEGA")[0].item()
p_vals, aucs = [], []

for i in range(5, len(random_seeds)):

    nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{i}")
    processing_dir_temp = os.path.join(nm_processing_dir_temp, method) + '/'

    p_val, auc = abnormal_probability(processing_dir_temp,
                                    nm_processing_dir_temp, 
                                    n_permutation=1000,
                                    site_id = omega_site_id,
                                    healthy_data_prefix = "estimate",
                                    patient_data_prefix = "clinicalpredict")
    
    p_vals.append(p_val); aucs.append(auc)

p_vals = pd.DataFrame(np.vstack(p_vals))
aucs = pd.DataFrame(np.vstack(aucs))

aucs.columns = biomarker_names
p_vals.columns = biomarker_names

**Boxplot of AUCs across biomarkers**

In [ ]:
box_plot_auc(aucs, 
    save_path=figures_dir,
    color=["orange", "teal", "olive", "tomato"],
    alpha=0.7,
    biomarkers_new_name=["Theta", "Alpha", "Beta", "Gamma"],)

**Extreme deviation statistics**

In [ ]:
processing_dir_temp = os.path.join(nm_processing_dir, method) 

df_c_pos, df_p_pos, df_c_neg, df_p_neg = plot_extreme_deviation(
                    base_path=nm_processing_dir, 
                    len_runs=10,
                    save_path=figures_dir,
                    healthy_prefix="estimate", 
                    site_id = [omega_site_id],
                    legend=["PD", "control"],
                    patient_prefix="clinicalpredict",
                    method=method,
                    new_col_name=['Theta', 'Alpha', 'Beta', 'Gamma'])

# 4. **Decoding heterogeneity in Parkinson's disease via theta-beta deviation profiles.**

In this section, we explore Parkinson's disease from a dimensional perspective by plotting the joint distribution of theta and beta deviation scores.

In [ ]:
processing_dir_temp = os.path.join(nm_processing_dir, method) + '/'
with open(os.path.join(processing_dir_temp, "Z_clinicalpredict.pkl"), "rb") as file:
    z_patient = pickle.load(file)
    z_patient.index = df_patients.index
    z_patient.columns = biomarker_names


z_scores_scatter_plot(X = list(z_patient.loc[:, "Adjusted_Canonical_Relative_PowerGamma_all"]),
                    Y = list(z_patient.loc[:, "Adjusted_Canonical_Relative_PowerBeta_all"]),
                    bands_name=["Gamma", "Beta"], 
                    lower_lim = -4.2,
                    upper_lim = 4.2,
                    ticks = [-4, -2, 0, 2, 4],
                    save_path=features_dir
                    )

# 5. **From growth charts to P-NOCs and I-NOCs.**

For the final step, we fit our proposed HBR models to the full dataset of healthy participants. From these normative models, we derive **growth charts** that capture the trajectory of spectral‐power changes across the lifespan. These charts then serve as the basis for generating both **Population‐level Neuro‑Oscillo Charts (P‑NOCs)** and **Individual‑level Neuro‑Oscillo Charts (I‑NOCs)**.


**Estimating Normative Models on the Full Dataset**

In [ ]:
nm_processing_dir_all = os.path.join(project_dir, 'NM', 'Run_all')
if not os.path.isdir(nm_processing_dir_all):
        os.mkdir(nm_processing_dir_all)

biomarker_names = hbr_data_split(df_healthy,
        nm_processing_dir_all,
        drop_nans=True,
        batch_effects=['sex', 'site'],
        random_seed=run_id,
        train_split=0.99,
        stratification_columns=["site"])

outputsuffix = '_estimateAll'

respfile = os.path.join(nm_processing_dir_all, 'y_train.pkl')
covfile = os.path.join(nm_processing_dir_all, 'x_train.pkl')
trbefile = os.path.join(nm_processing_dir_all, 'b_train.pkl')

testrespfile_path = os.path.join(nm_processing_dir_all, 'y_test.pkl')
testcovfile_path = os.path.join(nm_processing_dir_all, 'x_test.pkl')
tsbefile = os.path.join(nm_processing_dir_all, 'b_test.pkl')

processing_dir = os.path.join(nm_processing_dir_all, method) + '/'
nm_log_path = os.path.join(processing_dir, 'log') + '/'

if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)
if not os.path.isdir(nm_log_path):
    os.makedirs(nm_log_path)


execute_nm(processing_dir, 
            python_path,
            'NM', 
            covfile, 
            respfile, 
            batch_size, 
            memory, 
            duration, 
            alg='hbr', 
            log_path=nm_log_path, 
            binary=True, 
            testcovfile_path=testcovfile_path, 
            testrespfile_path=testrespfile_path,trbefile=trbefile, 
            tsbefile=tsbefile, 
            model_type=hbr_configs[method]['model_type'], 
            likelihood=hbr_configs[method]['likelihood'], 
            linear_sigma=hbr_configs[method]['linear_sigma'], 
            random_slope_mu=hbr_configs[method]['random_slope_mu'],
            linear_epsilon=hbr_configs[method]['linear_epsilon'], 
            linear_delta=hbr_configs[method]['linear_delta'], 
            savemodel='True', 
            inscaler=inscaler, 
            outscaler=outscaler, 
            outputsuffix=outputsuffix, 
            interactive='auto', 
            cluster_spec=cluster_spec)

In [ ]:
processing_dir_temp = os.path.join(nm_processing_dir_all, method) + '/'

collect_nm(processing_dir_temp, "NM", collect=True, binary=True, batch_size=1, outputsuffix=outputsuffix)

**Estimate centiles of variation for models, derived from the full dataset**

In [ ]:
outputsuffix='estimateAll'
# Plotting ranges
processing_dir_temp = os.path.join(nm_processing_dir_all, method)

q = estimate_centiles(processing_dir_temp, 
                    bio_num= len(biomarker_names), 
                    quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
                    age_range=[6, 80],
                    outputsuffix=outputsuffix)

**Growth charts**

In [ ]:
new_names = ["Theta", "Alpha", "Beta", "Gamma"]
processing_dir_temp = os.path.join(nm_processing_dir_all, method) 

plot_growthcharts(processing_dir_temp, 
                model_indices = list(range(len(new_names))), 
                biomarker_names = new_names, 
                site = None, 
                point_num  = 100, 
                number_of_sexs = 2, 
                num_of_sites = 6,
                centiles_name = ['5th', '25th', '50th', '75th', '95th'],
                colors = {"male": ['#4c061d', '#662333', '#803449', '#993d5e', '#b34e74'],
                        "female": ["#FF6F00", "#FF8C1A", "#FFA726", "#FFB74D", "#FFD54F"]},
                suffix=outputsuffix,
                save_path=features_dir)

**Individual NeuroOscilloCharts (I-NOCs)**

In [ ]:
processing_dir_temp = os.path.join(nm_processing_dir_all, method) 
q_path = os.path.join(processing_dir_temp, 
                    f"Quantiles_{outputsuffix}.pkl")

sub_index = ...
statistics = cal_stats_for_INOCs(q_path, 
                                biomarker_names, 
                                site_id=df_patients.loc[sub_index]["site"], 
                                sex_id=df_patients.loc[sub_index]["sex"], 
                                age=df_patients.loc[sub_index]["age"]*100,
                                num_of_datasets=len(datasets.keys()))

for i, name in enumerate(biomarker_names):

    if biomarker_names[i] == "Gamma": 
        max_value=0.2
    else: max_value=1

    plot_INOCs(sub_index = sub_index,
                observed_value = df_patients.loc[sub_index, name],
                q1 = statistics[name][1],
                q3 = statistics[name][3],
                percentile_5 = statistics[name][0],
                percentile_95 = statistics[name][4],
                percentile_50 = statistics[name][2],
                title="",
                max_value=max_value,
                show_legend=False,
                bio_name=["Theta", "Alpha", "Beta", "Gamma"][i],
                save_path=os.path.join(features_dir, "inocs_for_z_scores")
                )

**Population NeuroOscilloCharts (P-NOCs)**

In [ ]:
# # Calculateing Oscilograms
processing_dir_temp = os.path.join(nm_processing_dir_all, method) 
gender_ids = {'Male':0, 'Female':1}
frequency_band_model_ids = {'Theta':0, 'Alpha':1, 'Beta':2, 'Gamma':3}
quantiles_path= os.path.join(processing_dir_temp, f'Quantiles_{outputsuffix}.pkl')
oscilograms, age_slices = calculate_PNOCs(quantiles_path, gender_ids, frequency_band_model_ids, num_of_datasets=len(datasets.keys()))

plot_PNOCs(oscilograms, age_slices, save_path=features_dir)